In [3]:
import pandas as pd

#df1,(전기), df2(후기) 데이터 프레임
def cost_ratio(df1,df2):
    df1_sum = pd.pivot_table(df1,index='고객번호',values='구매금액',aggfunc='sum')
    df2_sum = pd.pivot_table(df2,index='고객번호',values='구매금액',aggfunc='sum')
    
    # 멀티인덱스 해제
    df1_sum = df1_sum.reset_index()
    df2_sum = df2_sum.reset_index()
    
    # 고객번호로 merge
    df = pd.merge(df1_sum,df2_sum,how='outer',on='고객번호')
    df.fillna(0,inplace=True)
    
    # 컬럼명 변경
    df.columns = ['고객번호','구매_1','구매_2']
    
    # 증감액 구하기
    df['증감액'] = df['구매_2'] - df['구매_1']
    
    # 등급 구하기
    def cust_level(x):
      if x >= 34350000: return 'VIP'
      elif x >= 20150000: return '최우수'
      elif x >= 8700000: return '우수'
      elif x == 0: return '없음'
      else: return '일반'
    
    # 증감 등급구하기
    def di_level(price1,price2,증감액):
        if (price1==0) & (price2 > 0): return '신규'
        elif (price1>0) & (price2 == 0): return '이탈'
        elif 증감액 > 0: return '증가'
        elif 증감액 == 0: return '유지'
        else: return '감소'
        
    df['level_1'] = df['구매_1'].apply(cust_level)
    df['level_2'] = df['구매_2'].apply(cust_level)
    df['di_level'] = df.apply(lambda x : di_level(x['구매_1'],x['구매_2'],x['증감액']),axis=1)
    
    
    level = pd.pivot_table(df,index=['level_1','level_2','di_level'],values='증감액',aggfunc=['sum','count'])
    level = level.reset_index()
    
    # di = decrease and increase, cust = 고객수
    level['di'] = level['sum']
    level['cust'] = level['count']
    
    # di 기여율 구하기
    level = level.drop(['sum','count'],axis=1)
    level['di_ratio'] =(level['di']/level['di'].sum()) *100
    
    # 고객 비율 구하기
    level['cust_ratio'] = (level['cust']/level['cust'].sum()) *100
    
    # 등급 합치기
    level['level'] = level[['level_1','level_2']].apply('_'.join,axis=1)
    level['level'] = level[['level','di_level']].apply('_'.join,axis=1)
    level_1 = level.drop(['level_1','level_2','di_level'],axis=1)
    
    # 등급 재배열
    level_1 = level_1[['level','cust','cust_ratio','di','di_ratio']]
    pd.options.display.float_format = '{:.5f}'.format
    return level_1

In [5]:
import pandas as pd

#df1,(전기), df2(후기) 데이터 프레임
def cost_ratio(df1,df2):
    df1_sum = pd.pivot_table(df1,index='고객번호',values='구매금액',aggfunc='sum')
    df2_sum = pd.pivot_table(df2,index='고객번호',values='구매금액',aggfunc='sum')
    
    # 멀티인덱스 해제
    df1_sum = df1_sum.reset_index()
    df2_sum = df2_sum.reset_index()
    
    # 고객번호로 merge
    df = pd.merge(df1_sum,df2_sum,how='outer',on='고객번호')
    df.fillna(0,inplace=True)
    
    # 컬럼명 변경
    df.columns = ['고객번호','구매_1','구매_2']
    
    # 증감액 구하기
    df['증감액'] = df['구매_2'] - df['구매_1']
    
    # 등급 구하기
    def cust_level(x):
      if x >= 34350000: return 'VIP'
      elif x >= 20150000: return '최우수'
      elif x >= 8700000: return '우수'
      elif x == 0: return '없음'
      else: return '일반'
    
    # 증감 등급구하기
    def di_level(price1,price2,증감액):
        if (price1==0) & (price2 > 0): return '신규'
        elif (price1>0) & (price2 == 0): return '이탈'
        elif 증감액 > 0: return '증가'
        elif 증감액 == 0: return '유지'
        else: return '감소'
        
    df['level_1'] = df['구매_1'].apply(cust_level)
    df['level_2'] = df['구매_2'].apply(cust_level)
#     df['di_level'] = df.apply(lambda x : di_level(x['구매_1'],x['구매_2'],x['증감액']),axis=1)
    
    
#     level = pd.pivot_table(df,index=['level_1','level_2','di_level'],values='증감액',aggfunc=['sum','count'])
#     level = level.reset_index()
    
#     # di = decrease and increase, cust = 고객수
#     level['di'] = level['sum']
#     level['cust'] = level['count']
    
#     # di 기여율 구하기
#     level = level.drop(['sum','count'],axis=1)
#     level['di_ratio'] =(level['di']/level['di'].sum()) *100
    
#     # 고객 비율 구하기
#     level['cust_ratio'] = (level['cust']/level['cust'].sum()) *100
    
#     # 등급 합치기
#     level['level'] = level[['level_1','level_2']].apply('_'.join,axis=1)
#     level['level'] = level[['level','di_level']].apply('_'.join,axis=1)
#     level_1 = level.drop(['level_1','level_2','di_level'],axis=1)
    
#     # 등급 재배열
#     level_1 = level_1[['level','cust','cust_ratio','di','di_ratio']]
#     pd.options.display.float_format = '{:.5f}'.format
    return df

In [4]:
prod = pd.read_csv('data/구매상품TR.txt',encoding='cp949')
prod['구매일자'] = pd.to_datetime(prod['구매일자'].astype(str))
prod['year'] = prod.구매일자.dt.year
prod['month'] = prod.구매일자.dt.month
y14_1 = prod[(prod['year']==2014)&(prod['month']<=6)]
y14_2 = prod[(prod['year']==2014)&(prod['month']>6)]

In [7]:
cost_ratio(y14_1, y14_2).level_1.value_counts()

일반     14348
우수      3169
최우수     1123
VIP      570
없음       128
Name: level_1, dtype: int64

In [15]:
cost_ratio(y14_1, y14_2).sort_values(by='di_ratio')

C:\anaconda3\envs\cakd3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,level,cust,cust_ratio,di,di_ratio
,,,,,
0,VIP_VIP_감소,155,0.80153,-5243983574.00000,-39.96823
16,일반_일반_감소,5496,28.42073,-4542170269.00000,-34.61920
20,최우수_우수_감소,424,2.19257,-3978155013.00000,-30.32042
4,VIP_최우수_감소,167,0.86358,-3431886686.00000,-26.15691
11,우수_일반_감소,601,3.10787,-2917783386.00000,-22.23856
9,우수_우수_감소,915,4.73162,-2594971719.00000,-19.77817
2,VIP_우수_감소,76,0.39301,-2577072873.00000,-19.64175
21,최우수_일반_감소,48,0.24822,-925673624.00000,-7.05523
22,최우수_최우수_감소,219,1.13249,-887248111.00000,-6.76237


In [5]:
y15_1 = prod[(prod['year']==2015)&(prod['month']<=6)]
y15_2 = prod[(prod['year']==2015)&(prod['month']>6)]
# y1415 = cost_ratio(y14_2, y15_1).sort_values(by='di_ratio')
# y1415['di_ratio'] = y1415['di_ratio']*-1
# y1415.sort_values(by='di_ratio')

In [17]:
cost_ratio(y15_1, y15_2).sort_values(by='di_ratio')

C:\anaconda3\envs\cakd3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,level,cust,cust_ratio,di,di_ratio
,,,,,
16,일반_일반_감소,6774,34.97522,-6415715841.00000,-72.58405
0,VIP_VIP_감소,189,0.97584,-4676491803.00000,-52.90738
20,최우수_우수_감소,426,2.19950,-4190256467.00000,-47.40637
11,우수_일반_감소,765,3.94981,-4073165013.00000,-46.08166
4,VIP_최우수_감소,193,0.99649,-3460214694.00000,-39.14706
2,VIP_우수_감소,87,0.44919,-2587242716.00000,-29.27071
9,우수_우수_감소,878,4.53325,-2584726307.00000,-29.24224
21,최우수_일반_감소,69,0.35626,-1278659626.00000,-14.46609
22,최우수_최우수_감소,206,1.06361,-819373677.00000,-9.26996


In [4]:
cost_ratio(y14_1, y15_2).sort_values(by='di_ratio')

C:\anaconda3\envs\cakd3\lib\site-packages\pandas\core\generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,level,cust,cust_ratio,di,di_ratio
,,,,,
18,일반_일반_감소,5574,28.76755,-6472567509.00000,-32.54601
13,우수_일반_감소,840,4.33526,-5181980171.00000,-26.05655
0,VIP_VIP_감소,131,0.67609,-4398842531.00000,-22.11870
3,VIP_일반_감소,33,0.17031,-4228654822.00000,-21.26295
22,최우수_우수_감소,403,2.07989,-4059838505.00000,-20.41409
4,VIP_최우수_감소,151,0.77931,-3300690614.00000,-16.59686
2,VIP_우수_감소,89,0.45933,-3024743337.00000,-15.20932
11,우수_우수_감소,748,3.86045,-2344531819.00000,-11.78901
23,최우수_일반_감소,96,0.49546,-1893838013.00000,-9.52279
